<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 경로 설정 
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier

In [66]:
df = pd.read_csv("landslide_asos_merged_final.csv")

In [68]:
df_sample = df.drop(['sd','sgg','umd','location','sum_cnt','sum_hpa'], axis=1)
df_sample

,stnId,stnNm,tm,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,landslide
0,115,울릉도,2011-01-01,0.8,-1.1,2.1,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0.0
1,130,울진,2011-01-01,-1.0,-7.0,2.3,0.0,0.0,15.25,8.5,20.2,12.3,6.0,40.0,79.0,0.0,0.0
2,136,안동,2011-01-01,-4.5,-11.0,2.9,0.0,0.0,0.00,0.0,7.4,4.4,2.1,41.0,58.8,0.0,0.0
3,137,상주,2011-01-01,-5.2,-9.9,0.5,0.0,0.0,0.00,0.0,9.8,5.3,3.3,36.0,50.0,0.0,0.0
4,138,포항,2011-01-01,-0.6,-6.2,5.2,0.0,0.0,0.00,0.0,11.1,4.7,2.8,39.0,48.1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102182,285,합천,2019-12-31,-1.5,-6.9,2.9,0.0,0.0,0.00,0.0,10.2,4.6,1.8,18.0,40.6,0.0,0.0
102183,288,밀양,2019-12-31,-1.1,-4.5,4.6,0.0,0.0,0.00,0.0,11.7,7.0,2.9,14.0,26.4,0.0,0.0
102184,289,산청,2019-12-31,-2.5,-5.1,1.2,0.0,0.0,0.00,0.0,21.0,10.4,5.8,22.0,34.9,0.0,0.0
102185,294,거제,2019-12-31,0.7,-1.5,6.7,0.0,0.0,0.00,0.0,10.3,4.7,2.4,25.0,40.1,0.0,0.0


In [71]:
df_sample = df_sample.dropna()
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101946 entries, 0 to 102186
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   stnId      101946 non-null  int64  
 1   stnNm      101946 non-null  object 
 2   tm         101946 non-null  object 
 3   avgTa      101946 non-null  float64
 4   minTa      101946 non-null  float64
 5   maxTa      101946 non-null  float64
 6   mi10MaxRn  101946 non-null  float64
 7   hr1MaxRn   101946 non-null  float64
 8   sumRnDur   101946 non-null  float64
 9   sumRn      101946 non-null  float64
 10  maxInsWs   101946 non-null  float64
 11  maxWs      101946 non-null  float64
 12  avgWs      101946 non-null  float64
 13  minRhm     101946 non-null  float64
 14  avgRhm     101946 non-null  float64
 15  n99Rn      101946 non-null  float64
 16  landslide  101946 non-null  float64
dtypes: float64(14), int64(1), object(2)
memory usage: 14.0+ MB


In [72]:
df_sample.columns

Index(['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'landslide'],
      dtype='object')

In [73]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df_sample[['avgTa', 'minTa', 'maxTa', 'mi10MaxRn','hr1MaxRn', 'sumRnDur', 'sumRn', 
       'maxInsWs', 'maxWs', 'avgWs', 'minRhm','avgRhm', 'n99Rn']], 
    df_sample['landslide'], 
    test_size=0.25, 
    stratify=df_sample['landslide'], 
    random_state=123456)

In [74]:
x_train

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn
98030,26.7,23.8,31.7,2.0,7.9,0.00,29.8,7.7,2.6,1.0,64.0,87.0,6.4
48656,10.0,8.5,11.4,1.0,6.0,0.00,15.5,11.9,7.0,2.4,70.0,84.3,0.0
10543,3.6,-1.6,9.2,0.0,0.0,0.00,0.0,6.0,3.6,2.1,30.0,65.6,0.0
73877,25.4,22.1,32.0,3.0,5.0,0.00,6.0,6.0,3.0,1.1,51.0,87.3,0.0
76691,16.1,14.2,20.5,0.0,0.0,0.00,0.0,4.9,3.6,0.9,62.0,84.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23927,0.9,-2.2,3.9,0.0,0.0,0.00,2.0,1.9,1.1,0.4,62.0,82.9,0.0
2809,10.3,-2.7,20.3,0.0,0.0,0.00,0.0,10.5,5.7,2.2,6.0,37.1,0.0
96061,23.2,17.1,30.1,0.0,0.0,0.00,0.0,6.6,3.7,1.2,43.0,69.0,0.0
7481,25.8,23.0,29.9,0.0,0.0,0.00,0.0,8.1,5.1,2.9,59.0,71.9,0.0


In [75]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=123456,
                       verbose=0, warm_start=False)

In [76]:
from sklearn.metrics import accuracy_score

predicted = rf.predict(x_test)
accuracy = accuracy_score(y_test, predicted)

print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

Out-of-bag score estimate: 1.0
Mean accuracy score: 1.0


In [59]:
predicted

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from sklearn.metrics import confusion_matrix

cm = pd.DataFrame(confusion_matrix(y_test, predicted))
sns.heatmap(cm, annot=True)